In [11]:
## imports
import os
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
device = 'cpu'
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    device = 'cuda'

CUDA is available!  Training on GPU ...


In [13]:
data_transform =  transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [14]:
data_dir = 'food-101/images'
batch_size = 100
num_workers = 0
dataset = datasets.ImageFolder(
        data_dir, 
        transform=data_transform)
print(len(dataset),"images")
## split data between train and test. Total of 101,000 images 
## we use 100,000 for aining and 1,000 for testing
train_data,test_data=torch.utils.data.random_split(dataset,(9000,1000))
# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)

10000 images


In [15]:

densenet = models.densenet161(pretrained=True) ## load the pretrained model
#print(densenet)

In [16]:

# Freeze training for all "features" layers
for param in densenet.features.parameters():
    param.requires_grad = False

In [17]:

import torch.nn as nn
n_inputs = densenet.classifier.in_features
last_layer = nn.Linear(n_inputs,10)
densenet.classifier = last_layer
# if GPU is available, move the model to GPU
if train_on_gpu:
    densenet.cuda()
# check to see that your last layer produces the expected number of outputs
print(densenet.classifier.out_features)

10


In [18]:

import torch.optim as optim
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.Adam(densenet.classifier.parameters(), lr=0.001)


In [19]:

## function to train the network
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss /= len(train_loader.dataset)
    print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch, correct, len(train_loader.dataset),
                100. * correct / len(train_loader.dataset), train_loss))

## function to train the network
def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test loss: {:.4f}, Test Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [20]:
# number of epochs to train the model
n_epochs = 7
for epoch in range(1, n_epochs+1):
    
    train (densenet, device, train_loader, optimizer, criterion, epoch)
    test (densenet, device, criterion, test_loader)
    
    print ('\n')
            
print ("Done training")


Epoch: 1 , Training Accuracy: 6624/9000 (74%) Training Loss: 0.010990
Test loss: 0.0058, Test Accuracy: 861/1000 (86%)



Epoch: 2 , Training Accuracy: 7871/9000 (87%) Training Loss: 0.005144
Test loss: 0.0044, Test Accuracy: 880/1000 (88%)



Epoch: 3 , Training Accuracy: 8004/9000 (89%) Training Loss: 0.004133
Test loss: 0.0038, Test Accuracy: 898/1000 (90%)



Epoch: 4 , Training Accuracy: 8118/9000 (90%) Training Loss: 0.003564
Test loss: 0.0035, Test Accuracy: 900/1000 (90%)



Epoch: 5 , Training Accuracy: 8172/9000 (91%) Training Loss: 0.003289
Test loss: 0.0033, Test Accuracy: 904/1000 (90%)



Epoch: 6 , Training Accuracy: 8239/9000 (92%) Training Loss: 0.003003
Test loss: 0.0032, Test Accuracy: 907/1000 (91%)



Epoch: 7 , Training Accuracy: 8294/9000 (92%) Training Loss: 0.002789
Test loss: 0.0032, Test Accuracy: 911/1000 (91%)



Done training


In [21]:

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
        outputs = densenet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 91 %


In [22]:
torch.save(densenet.state_dict(), 'densenet-91.pwf')